In [73]:
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from darksky import forecast
import pickle
from sklearn.preprocessing import MinMaxScaler
import time

In [34]:
def convert_time(str_time):
    return dt.strptime(str_time, "%Y-%m-%d %H:%M:%S")

def first_cleansing_step(data):
    data['Time'] = data['Time'].apply(convert_time)
    return data

def filter_weather(weather):
    return weather

In [35]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [36]:
key = 'c9f5b49eab51e5a3a98bae35a9bcbb88'
weather_cache = {}

In [37]:
def get_weather_data(prediction_time):
#     print("Getting response for", prediction_time)
    if prediction_time in weather_cache:
        return weather_cache[prediction_time]
    sim_center = key, 35.042677, -85.299246
    sim_center = forecast(*sim_center, time=prediction_time.isoformat())
    weather_cache[prediction_time] = sim_center['currently']
#     print("Got response for", prediction_time)
    return sim_center['currently']

In [38]:
 with open('weather_cache.pkl', 'rb') as f:
    weather_cache = pickle.load(f)

In [39]:
def convert_time(str_time):
    return dt.strptime(str_time, "%Y-%m-%d %H:%M:%S")

def first_cleansing_step(data):
    data['Time'] = data['Time'].apply(convert_time)
    return data

def filter_weather(weather):
    return weather

In [40]:
pd.DataFrame(weather_cache).to_csv("weather_cache.csv")

In [41]:
def add_weather(dataset):
    res = []
    for _, item in dataset.iterrows():
        weather_data = get_weather_data(item["Time"])
        res.append({**item, **weather_data})
    return res

In [60]:
def preprocess_data(data):
    for column in data.columns:
        if data[column].dtype == object:
            data[column] = data[column].astype('category').cat.codes
    return data

In [97]:
traffic_data = pd.read_csv("vehicles_april_2019.csv", sep=";")
pollution_data = pd.read_csv("pm_2.5_april_2019.csv", sep=";")
traffic_data = first_cleansing_step(traffic_data)
pollution_data = first_cleansing_step(pollution_data)

In [98]:
w_pollution_data = add_weather(pollution_data)
w_pollution_data = pd.DataFrame(w_pollution_data)

In [99]:
w_pollution_data.head()

,Time,apparentTemperature,cloudCover,dewPoint,humidity,icon,mlk-central,mlk-douglas,mlk-georgia,mlk-houston,...,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,2019-03-18 16:00:00,55.39000,0.00000,12.03000,0.18000,clear-day,0.44000,0.09000,0.33000,nan,...,NaN,1025.42000,Clear,55.39000,1552939200,5.00000,5.53000,348.00000,19.23000,7.35000
1,2019-03-18 16:05:00,55.43000,0.00000,11.91000,0.17000,clear-day,0.50333,0.11750,0.33000,nan,...,NaN,1025.38000,Clear,55.43000,1552939500,5.00000,5.53000,349.00000,19.20000,7.58000
2,2019-03-18 16:10:00,55.47000,0.00000,11.78000,0.17000,clear-day,0.38500,0.38500,0.33000,nan,...,NaN,1025.34000,Clear,55.47000,1552939800,5.00000,5.53000,349.00000,19.18000,7.81000
3,2019-03-18 16:15:00,55.51000,0.00000,11.65000,0.17000,clear-day,0.45250,0.36750,0.33000,nan,...,NaN,1025.31000,Clear,55.51000,1552940100,4.00000,5.53000,350.00000,19.15000,8.03000
4,2019-03-18 16:20:00,55.55000,0.00000,11.52000,0.17000,clear-day,0.25250,0.20000,0.31000,nan,...,NaN,1025.27000,Clear,55.55000,1552940400,4.00000,5.53000,351.00000,19.13000,8.26000


In [100]:
w_pollution_data.describe()

,apparentTemperature,cloudCover,dewPoint,humidity,mlk-central,mlk-douglas,mlk-georgia,mlk-houston,mlk-lindsay,mlk-magnolia,...,precipIntensity,precipProbability,pressure,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
count,8637.00000,8603.00000,8637.00000,8637.00000,8617.00000,8637.00000,3243.00000,2773.00000,7442.00000,8638.00000,...,8592.00000,8592.00000,8637.00000,8637.00000,8641.00000,8623.00000,8618.00000,8634.00000,8605.00000,8634.00000
mean,59.34940,0.45520,41.19896,0.54066,1.18135,0.74795,0.84730,1.30821,1.80584,0.89870,...,0.00424,0.06706,1018.70838,59.74280,1554235200.00000,1.60594,5.44287,187.59937,10.53939,4.58143
std,12.11703,0.36083,13.38535,0.19724,0.85177,0.60781,0.57504,0.87090,2.01671,0.61219,...,0.02272,0.21420,6.22787,11.41573,748375.85143,2.36633,1.63140,108.25854,5.38784,3.15821
min,29.69000,0.00000,10.43000,0.13000,0.00000,0.00000,0.00000,0.11000,0.04000,0.02000,...,0.00000,0.00000,998.23000,33.21000,1552939200.00000,0.00000,1.15000,0.00000,1.51000,0.02000
25%,50.69000,0.03000,31.00000,0.38000,0.56500,0.37500,0.40333,0.64500,0.79500,0.44500,...,0.00000,0.00000,1014.70000,50.69000,1553587200.00000,0.00000,4.33000,104.25000,6.37000,1.94000
50%,60.14000,0.46000,39.68000,0.55000,0.94750,0.58667,0.68250,1.10750,1.34500,0.73250,...,0.00000,0.00000,1019.31000,60.14000,1554235200.00000,0.00000,5.53000,191.00000,9.40000,3.80000
75%,68.81000,0.75000,53.72000,0.70000,1.58000,0.92250,1.15000,1.74500,2.41750,1.18750,...,0.00000,0.00000,1023.63000,68.64000,1554883200.00000,3.00000,6.65000,296.00000,13.49000,6.70000
max,85.55000,1.00000,65.70000,0.94000,16.25750,13.86000,7.18250,7.25500,128.40750,8.13667,...,0.42700,1.00000,1030.78000,85.55000,1555531200.00000,10.00000,10.00000,359.00000,35.73000,16.09000


In [131]:
min_time = min(w_pollution_data['Time'])
max_time = max(w_pollution_data['Time'])
print("Time between {} and {} which is = {} days".format(min_time, max_time, (max_time - min_time).days))

Time between 2019-03-18 16:00:00 and 2019-04-17 16:00:00 which is = 30 days


In [102]:
w_pollution_data_frame = preprocess_data(w_pollution_data)
w_pollution_data_frame.to_csv("weather_pollution.csv")

In [103]:
w_pollution_data_frame = preprocess_data(w_pollution_data)
w_pollution_data_frame.head()

,Time,apparentTemperature,cloudCover,dewPoint,humidity,icon,mlk-central,mlk-douglas,mlk-georgia,mlk-houston,...,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,2019-03-18 16:00:00,55.39000,0.00000,12.03000,0.18000,0,0.44000,0.09000,0.33000,nan,...,-1,1025.42000,2,55.39000,1552939200,5.00000,5.53000,348.00000,19.23000,7.35000
1,2019-03-18 16:05:00,55.43000,0.00000,11.91000,0.17000,0,0.50333,0.11750,0.33000,nan,...,-1,1025.38000,2,55.43000,1552939500,5.00000,5.53000,349.00000,19.20000,7.58000
2,2019-03-18 16:10:00,55.47000,0.00000,11.78000,0.17000,0,0.38500,0.38500,0.33000,nan,...,-1,1025.34000,2,55.47000,1552939800,5.00000,5.53000,349.00000,19.18000,7.81000
3,2019-03-18 16:15:00,55.51000,0.00000,11.65000,0.17000,0,0.45250,0.36750,0.33000,nan,...,-1,1025.31000,2,55.51000,1552940100,4.00000,5.53000,350.00000,19.15000,8.03000
4,2019-03-18 16:20:00,55.55000,0.00000,11.52000,0.17000,0,0.25250,0.20000,0.31000,nan,...,-1,1025.27000,2,55.55000,1552940400,4.00000,5.53000,351.00000,19.13000,8.26000


In [105]:
traffic_data.head()

,Time,mlk-peeples-cam-3 car,mlk-peeples-cam-1 car,mlk-magnolia-cam-2 car,mlk-lindsay-cam-2 car,mlk-houston-cam-1 car,mlk-georgia-cam-3 car,mlk-georgia-cam-1 car,mlk-douglas-cam-2 car,mlk-douglas-cam-1 car,mlk-central-cam-2 car
0,2019-03-18 16:02:00,26,22,32,0,0,0,0,26,0,27
1,2019-03-18 16:03:00,17,9,13,0,0,0,0,26,0,64
2,2019-03-18 16:04:00,32,14,27,0,0,0,0,32,0,66
3,2019-03-18 16:05:00,41,25,27,0,0,0,0,17,0,58
4,2019-03-18 16:06:00,40,24,21,0,0,0,0,35,0,58


In [138]:
traffic_data.describe()

,mlk-peeples-cam-3 car,mlk-peeples-cam-1 car,mlk-magnolia-cam-2 car,mlk-lindsay-cam-2 car,mlk-houston-cam-1 car,mlk-georgia-cam-3 car,mlk-georgia-cam-1 car,mlk-douglas-cam-2 car,mlk-douglas-cam-1 car,mlk-central-cam-2 car
count,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000,43201.00000
mean,7.75760,6.09627,4.14305,9.68461,12.05810,14.82801,13.42985,8.02933,8.02185,16.14259
std,9.94217,7.57456,6.45931,12.15406,13.53309,16.26164,15.46341,9.65707,11.07632,16.37290
min,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000
50%,4.00000,3.00000,1.00000,4.00000,7.00000,8.00000,7.00000,4.00000,3.00000,12.00000
75%,11.00000,9.00000,6.00000,17.00000,21.00000,27.00000,23.00000,13.00000,12.00000,25.00000
max,81.00000,79.00000,65.00000,77.00000,85.00000,87.00000,88.00000,67.00000,86.00000,99.00000


In [121]:
traffic_data = preprocess_data(traffic_data)

In [130]:
min_time = min(traffic_data['Time'])
max_time = max(traffic_data['Time'])
print("Time between {} and {} which is = {} days".format(min_time, max_time, (max_time - min_time).days))

Time between 2019-03-18 16:02:00 and 2019-04-17 16:02:00 which is = 30 days


In [133]:
pollution_data_times = w_pollution_data["Time"]
traffic_times = traffic_data["Time"]

In [137]:
for a,b in zip(traffic_times, pollution_data_times[:len(traffic_times)]):
    print(a,"----->", b)

2019-03-18 16:02:00 -----> 2019-03-18 16:00:00
2019-03-18 16:03:00 -----> 2019-03-18 16:05:00
2019-03-18 16:04:00 -----> 2019-03-18 16:10:00
2019-03-18 16:05:00 -----> 2019-03-18 16:15:00
2019-03-18 16:06:00 -----> 2019-03-18 16:20:00
2019-03-18 16:07:00 -----> 2019-03-18 16:25:00
2019-03-18 16:08:00 -----> 2019-03-18 16:30:00
2019-03-18 16:09:00 -----> 2019-03-18 16:35:00
2019-03-18 16:10:00 -----> 2019-03-18 16:40:00
2019-03-18 16:11:00 -----> 2019-03-18 16:45:00
2019-03-18 16:12:00 -----> 2019-03-18 16:50:00
2019-03-18 16:13:00 -----> 2019-03-18 16:55:00
2019-03-18 16:14:00 -----> 2019-03-18 17:00:00
2019-03-18 16:15:00 -----> 2019-03-18 17:05:00
2019-03-18 16:16:00 -----> 2019-03-18 17:10:00
2019-03-18 16:17:00 -----> 2019-03-18 17:15:00
2019-03-18 16:18:00 -----> 2019-03-18 17:20:00
2019-03-18 16:19:00 -----> 2019-03-18 17:25:00
2019-03-18 16:20:00 -----> 2019-03-18 17:30:00
2019-03-18 16:21:00 -----> 2019-03-18 17:35:00
2019-03-18 16:22:00 -----> 2019-03-18 17:40:00
2019-03-18 16

2019-03-19 06:18:00 -----> 2019-03-21 15:20:00
2019-03-19 06:19:00 -----> 2019-03-21 15:25:00
2019-03-19 06:20:00 -----> 2019-03-21 15:30:00
2019-03-19 06:21:00 -----> 2019-03-21 15:35:00
2019-03-19 06:22:00 -----> 2019-03-21 15:40:00
2019-03-19 06:23:00 -----> 2019-03-21 15:45:00
2019-03-19 06:24:00 -----> 2019-03-21 15:50:00
2019-03-19 06:25:00 -----> 2019-03-21 15:55:00
2019-03-19 06:26:00 -----> 2019-03-21 16:00:00
2019-03-19 06:27:00 -----> 2019-03-21 16:05:00
2019-03-19 06:28:00 -----> 2019-03-21 16:10:00
2019-03-19 06:29:00 -----> 2019-03-21 16:15:00
2019-03-19 06:30:00 -----> 2019-03-21 16:20:00
2019-03-19 06:31:00 -----> 2019-03-21 16:25:00
2019-03-19 06:32:00 -----> 2019-03-21 16:30:00
2019-03-19 06:33:00 -----> 2019-03-21 16:35:00
2019-03-19 06:34:00 -----> 2019-03-21 16:40:00
2019-03-19 06:35:00 -----> 2019-03-21 16:45:00
2019-03-19 06:36:00 -----> 2019-03-21 16:50:00
2019-03-19 06:37:00 -----> 2019-03-21 16:55:00
2019-03-19 06:38:00 -----> 2019-03-21 17:00:00
2019-03-19 06

2019-03-19 19:24:00 -----> 2019-03-24 08:50:00
2019-03-19 19:25:00 -----> 2019-03-24 08:55:00
2019-03-19 19:26:00 -----> 2019-03-24 09:00:00
2019-03-19 19:27:00 -----> 2019-03-24 09:05:00
2019-03-19 19:28:00 -----> 2019-03-24 09:10:00
2019-03-19 19:29:00 -----> 2019-03-24 09:15:00
2019-03-19 19:30:00 -----> 2019-03-24 09:20:00
2019-03-19 19:31:00 -----> 2019-03-24 09:25:00
2019-03-19 19:32:00 -----> 2019-03-24 09:30:00
2019-03-19 19:33:00 -----> 2019-03-24 09:35:00
2019-03-19 19:34:00 -----> 2019-03-24 09:40:00
2019-03-19 19:35:00 -----> 2019-03-24 09:45:00
2019-03-19 19:36:00 -----> 2019-03-24 09:50:00
2019-03-19 19:37:00 -----> 2019-03-24 09:55:00
2019-03-19 19:38:00 -----> 2019-03-24 10:00:00
2019-03-19 19:39:00 -----> 2019-03-24 10:05:00
2019-03-19 19:40:00 -----> 2019-03-24 10:10:00
2019-03-19 19:41:00 -----> 2019-03-24 10:15:00
2019-03-19 19:42:00 -----> 2019-03-24 10:20:00
2019-03-19 19:43:00 -----> 2019-03-24 10:25:00
2019-03-19 19:44:00 -----> 2019-03-24 10:30:00
2019-03-19 19

2019-03-20 07:17:00 -----> 2019-03-26 20:15:00
2019-03-20 07:18:00 -----> 2019-03-26 20:20:00
2019-03-20 07:19:00 -----> 2019-03-26 20:25:00
2019-03-20 07:20:00 -----> 2019-03-26 20:30:00
2019-03-20 07:21:00 -----> 2019-03-26 20:35:00
2019-03-20 07:22:00 -----> 2019-03-26 20:40:00
2019-03-20 07:23:00 -----> 2019-03-26 20:45:00
2019-03-20 07:24:00 -----> 2019-03-26 20:50:00
2019-03-20 07:25:00 -----> 2019-03-26 20:55:00
2019-03-20 07:26:00 -----> 2019-03-26 21:00:00
2019-03-20 07:27:00 -----> 2019-03-26 21:05:00
2019-03-20 07:28:00 -----> 2019-03-26 21:10:00
2019-03-20 07:29:00 -----> 2019-03-26 21:15:00
2019-03-20 07:30:00 -----> 2019-03-26 21:20:00
2019-03-20 07:31:00 -----> 2019-03-26 21:25:00
2019-03-20 07:32:00 -----> 2019-03-26 21:30:00
2019-03-20 07:33:00 -----> 2019-03-26 21:35:00
2019-03-20 07:34:00 -----> 2019-03-26 21:40:00
2019-03-20 07:35:00 -----> 2019-03-26 21:45:00
2019-03-20 07:36:00 -----> 2019-03-26 21:50:00
2019-03-20 07:37:00 -----> 2019-03-26 21:55:00
2019-03-20 07

2019-03-20 18:24:00 -----> 2019-03-29 03:50:00
2019-03-20 18:25:00 -----> 2019-03-29 03:55:00
2019-03-20 18:26:00 -----> 2019-03-29 04:00:00
2019-03-20 18:27:00 -----> 2019-03-29 04:05:00
2019-03-20 18:28:00 -----> 2019-03-29 04:10:00
2019-03-20 18:29:00 -----> 2019-03-29 04:15:00
2019-03-20 18:30:00 -----> 2019-03-29 04:20:00
2019-03-20 18:31:00 -----> 2019-03-29 04:25:00
2019-03-20 18:32:00 -----> 2019-03-29 04:30:00
2019-03-20 18:33:00 -----> 2019-03-29 04:35:00
2019-03-20 18:34:00 -----> 2019-03-29 04:40:00
2019-03-20 18:35:00 -----> 2019-03-29 04:45:00
2019-03-20 18:36:00 -----> 2019-03-29 04:50:00
2019-03-20 18:37:00 -----> 2019-03-29 04:55:00
2019-03-20 18:38:00 -----> 2019-03-29 05:00:00
2019-03-20 18:39:00 -----> 2019-03-29 05:05:00
2019-03-20 18:40:00 -----> 2019-03-29 05:10:00
2019-03-20 18:41:00 -----> 2019-03-29 05:15:00
2019-03-20 18:42:00 -----> 2019-03-29 05:20:00
2019-03-20 18:43:00 -----> 2019-03-29 05:25:00
2019-03-20 18:44:00 -----> 2019-03-29 05:30:00
2019-03-20 18

...

2019-03-22 09:17:00 -----> 2019-04-06 06:15:00
2019-03-22 09:18:00 -----> 2019-04-06 06:20:00
2019-03-22 09:19:00 -----> 2019-04-06 06:25:00
2019-03-22 09:20:00 -----> 2019-04-06 06:30:00
2019-03-22 09:21:00 -----> 2019-04-06 06:35:00
2019-03-22 09:22:00 -----> 2019-04-06 06:40:00
2019-03-22 09:23:00 -----> 2019-04-06 06:45:00
2019-03-22 09:24:00 -----> 2019-04-06 06:50:00
2019-03-22 09:25:00 -----> 2019-04-06 06:55:00
2019-03-22 09:26:00 -----> 2019-04-06 07:00:00
2019-03-22 09:27:00 -----> 2019-04-06 07:05:00
2019-03-22 09:28:00 -----> 2019-04-06 07:10:00
2019-03-22 09:29:00 -----> 2019-04-06 07:15:00
2019-03-22 09:30:00 -----> 2019-04-06 07:20:00
2019-03-22 09:31:00 -----> 2019-04-06 07:25:00
2019-03-22 09:32:00 -----> 2019-04-06 07:30:00
2019-03-22 09:33:00 -----> 2019-04-06 07:35:00
2019-03-22 09:34:00 -----> 2019-04-06 07:40:00
2019-03-22 09:35:00 -----> 2019-04-06 07:45:00
2019-03-22 09:36:00 -----> 2019-04-06 07:50:00
2019-03-22 09:37:00 -----> 2019-04-06 07:55:00
2019-03-22 09

2019-03-22 21:48:00 -----> 2019-04-08 20:50:00
2019-03-22 21:49:00 -----> 2019-04-08 20:55:00
2019-03-22 21:50:00 -----> 2019-04-08 21:00:00
2019-03-22 21:51:00 -----> 2019-04-08 21:05:00
2019-03-22 21:52:00 -----> 2019-04-08 21:10:00
2019-03-22 21:53:00 -----> 2019-04-08 21:15:00
2019-03-22 21:54:00 -----> 2019-04-08 21:20:00
2019-03-22 21:55:00 -----> 2019-04-08 21:25:00
2019-03-22 21:56:00 -----> 2019-04-08 21:30:00
2019-03-22 21:57:00 -----> 2019-04-08 21:35:00
2019-03-22 21:58:00 -----> 2019-04-08 21:40:00
2019-03-22 21:59:00 -----> 2019-04-08 21:45:00
2019-03-22 22:00:00 -----> 2019-04-08 21:50:00
2019-03-22 22:01:00 -----> 2019-04-08 21:55:00
2019-03-22 22:02:00 -----> 2019-04-08 22:00:00
2019-03-22 22:03:00 -----> 2019-04-08 22:05:00
2019-03-22 22:04:00 -----> 2019-04-08 22:10:00
2019-03-22 22:05:00 -----> 2019-04-08 22:15:00
2019-03-22 22:06:00 -----> 2019-04-08 22:20:00
2019-03-22 22:07:00 -----> 2019-04-08 22:25:00
2019-03-22 22:08:00 -----> 2019-04-08 22:30:00
2019-03-22 22

2019-03-23 10:16:00 -----> 2019-04-11 11:10:00
2019-03-23 10:17:00 -----> 2019-04-11 11:15:00
2019-03-23 10:18:00 -----> 2019-04-11 11:20:00
2019-03-23 10:19:00 -----> 2019-04-11 11:25:00
2019-03-23 10:20:00 -----> 2019-04-11 11:30:00
2019-03-23 10:21:00 -----> 2019-04-11 11:35:00
2019-03-23 10:22:00 -----> 2019-04-11 11:40:00
2019-03-23 10:23:00 -----> 2019-04-11 11:45:00
2019-03-23 10:24:00 -----> 2019-04-11 11:50:00
2019-03-23 10:25:00 -----> 2019-04-11 11:55:00
2019-03-23 10:26:00 -----> 2019-04-11 12:00:00
2019-03-23 10:27:00 -----> 2019-04-11 12:05:00
2019-03-23 10:28:00 -----> 2019-04-11 12:10:00
2019-03-23 10:29:00 -----> 2019-04-11 12:15:00
2019-03-23 10:30:00 -----> 2019-04-11 12:20:00
2019-03-23 10:31:00 -----> 2019-04-11 12:25:00
2019-03-23 10:32:00 -----> 2019-04-11 12:30:00
2019-03-23 10:33:00 -----> 2019-04-11 12:35:00
2019-03-23 10:34:00 -----> 2019-04-11 12:40:00
2019-03-23 10:35:00 -----> 2019-04-11 12:45:00
2019-03-23 10:36:00 -----> 2019-04-11 12:50:00
2019-03-23 10

2019-03-24 00:09:00 -----> 2019-04-14 08:35:00
2019-03-24 00:10:00 -----> 2019-04-14 08:40:00
2019-03-24 00:11:00 -----> 2019-04-14 08:45:00
2019-03-24 00:12:00 -----> 2019-04-14 08:50:00
2019-03-24 00:13:00 -----> 2019-04-14 08:55:00
2019-03-24 00:14:00 -----> 2019-04-14 09:00:00
2019-03-24 00:15:00 -----> 2019-04-14 09:05:00
2019-03-24 00:16:00 -----> 2019-04-14 09:10:00
2019-03-24 00:17:00 -----> 2019-04-14 09:15:00
2019-03-24 00:18:00 -----> 2019-04-14 09:20:00
2019-03-24 00:19:00 -----> 2019-04-14 09:25:00
2019-03-24 00:20:00 -----> 2019-04-14 09:30:00
2019-03-24 00:21:00 -----> 2019-04-14 09:35:00
2019-03-24 00:22:00 -----> 2019-04-14 09:40:00
2019-03-24 00:23:00 -----> 2019-04-14 09:45:00
2019-03-24 00:24:00 -----> 2019-04-14 09:50:00
2019-03-24 00:25:00 -----> 2019-04-14 09:55:00
2019-03-24 00:26:00 -----> 2019-04-14 10:00:00
2019-03-24 00:27:00 -----> 2019-04-14 10:05:00
2019-03-24 00:28:00 -----> 2019-04-14 10:10:00
2019-03-24 00:29:00 -----> 2019-04-14 10:15:00
2019-03-24 00

2019-03-24 14:03:00 -----> 2019-04-17 06:05:00
2019-03-24 14:04:00 -----> 2019-04-17 06:10:00
2019-03-24 14:05:00 -----> 2019-04-17 06:15:00
2019-03-24 14:06:00 -----> 2019-04-17 06:20:00
2019-03-24 14:07:00 -----> 2019-04-17 06:25:00
2019-03-24 14:08:00 -----> 2019-04-17 06:30:00
2019-03-24 14:09:00 -----> 2019-04-17 06:35:00
2019-03-24 14:10:00 -----> 2019-04-17 06:40:00
2019-03-24 14:11:00 -----> 2019-04-17 06:45:00
2019-03-24 14:12:00 -----> 2019-04-17 06:50:00
2019-03-24 14:13:00 -----> 2019-04-17 06:55:00
2019-03-24 14:14:00 -----> 2019-04-17 07:00:00
2019-03-24 14:15:00 -----> 2019-04-17 07:05:00
2019-03-24 14:16:00 -----> 2019-04-17 07:10:00
2019-03-24 14:17:00 -----> 2019-04-17 07:15:00
2019-03-24 14:18:00 -----> 2019-04-17 07:20:00
2019-03-24 14:19:00 -----> 2019-04-17 07:25:00
2019-03-24 14:20:00 -----> 2019-04-17 07:30:00
2019-03-24 14:21:00 -----> 2019-04-17 07:35:00
2019-03-24 14:22:00 -----> 2019-04-17 07:40:00
2019-03-24 14:23:00 -----> 2019-04-17 07:45:00
2019-03-24 14